<a href="https://colab.research.google.com/github/pranabsarkar/comment_reviews_nlp/blob/master/Final_Niki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


# Importing the libraries.

In [0]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import math
import re
from textblob import TextBlob
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
from sklearn import *
from sklearn.ensemble import RandomForestClassifier
import smote_variants as sv


## Loading the Data

In [0]:
df_train=pd.read_csv("/content/drive/My Drive/Colab Notebooks/niki/train.csv")
df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/niki/test.csv")

## Lets have a look into the training Data

In [0]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5693 entries, 0 to 5692
Data columns (total 6 columns):
id                  5693 non-null object
App Version Code    4554 non-null float64
App Version Name    4554 non-null float64
Review Text         5692 non-null object
Review Title        602 non-null object
Star Rating         5693 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 266.9+ KB


In [0]:
df_train['Star Rating'].value_counts()

5    2923
1    1788
4     611
3     217
2     154
Name: Star Rating, dtype: int64

In [0]:
df_train.tail(5)

,id,App Version Code,App Version Name,Review Text,Review Title,Star Rating
5688,6a486dfa-10ec-46c3-be48-da4296ff3690,69.0,2.28,Good,NaN,4
5689,9c6745d7-64bd-431a-9861-0737db52eadc,69.0,2.28,Easy to use,NaN,4
5690,2289f5fd-5556-49f8-ab4f-ba5e99a1a50f,68.0,2.27,I'm already a Lazypay account holder using sam...,NaN,2
5691,c11b87db-6e1d-4b52-ae1e-d759809ef2f0,75.0,2.34,"It's so easy to use, I'm just using and fallin...",NaN,5
5692,0a4c9a2a-8510-4e1d-a5ba-bd30afbac09a,NaN,NaN,Good,NaN,5


#Note:

After the analysis I have noticed the following:

1.   There are many missing values.
2.   The data is not balanced.
3.   The text data(Review Text) is not clean.

---

Thus, I will use the entire data as well as create some synthetic data to balance the dataset for training my model.

The steps I will follow :

1.   Data Processing
2.   Creating the synthetic data to balance the dataset and training the model.
3.   Creating the Submission File.





# 1. Data Processing

In [0]:
def clean_text(text):
  p=' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
  return p.lower()

In [0]:
# Using sentiment analysis(polarity) to score the review text and to categorize the review title. 

def layer1_preprocess(df):
  main_df=df
  main_df=main_df.dropna(subset = ['Review Text'])
  main_df['Review Title']= main_df['Review Title'].fillna('pranab')
  main_df['senti']=0.0
  main_df['senti_2']='wow'
  for i in range(len(main_df)):
    analysis = TextBlob(clean_text(main_df['Review Text'].iloc[i]))
    nrr=analysis.sentiment.polarity
    main_df['senti'].iloc[i]=nrr
    if main_df['Review Title'].iloc[i]=='pranab':
      main_df['senti_2'].iloc[i]='empty3'
    else:
      analysis = TextBlob(clean_text(str(main_df['Review Title'].iloc[i])))
      nrr=analysis.sentiment.polarity
      if nrr>0.0:
        main_df['senti_2'].iloc[i]='happy'
      elif nrr<0.0:
        main_df['senti_2'].iloc[i]='sad'
      else:
        main_df['senti_2'].iloc[i]='neutral'   
  return main_df
    

  

In [0]:
# This Process will take some minutes..
p1=layer1_preprocess(df_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [0]:
# This Process will take some minutes..
p2=layer1_preprocess(df_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [0]:
p1.head()

,id,App Version Code,App Version Name,Review Text,Review Title,Star Rating,senti,senti_2
0,b817b0f0-a2f8-4c9d-a5f6-d3fbf8b1d7e9,NaN,NaN,Very bad wallet balance not use.,pranab,1,-0.91,empty3
1,c7844e8f-56c1-487b-ae3e-df2fdf4c1767,43.0,2.30,Froud app i recharge 199 but not done also sen...,pranab,1,0.00,empty3
2,93ff57f7-9e02-4fa9-b779-3db82b8af0a4,52.0,2.12,Waste to write comments also,Waste no use,2,-0.20,sad
3,df2dcdef-c09a-4f35-afab-e1231d3fec9a,62.0,2.21,Nice apo,pranab,5,0.60,empty3
4,11f8f968-4cec-4424-8427-9709ab05b5be,69.0,2.28,Good nice app,pranab,5,0.65,empty3


In [0]:
p2.head()

,id,App Version Code,App Version Name,Review Text,Review Title,senti,senti_2
0,bdcb3129-afc1-4608-825f-558fe9c17e2b,78.0,2.37,"Awesome app, all should use it",pranab,1.000000,empty3
1,7518d5c8-5e35-45e2-b29d-cf0996ba9e2f,75.0,2.34,how can i stop notification,pranab,0.000000,empty3
2,b21ca913-ba44-471b-91e1-aaf035379a84,NaN,NaN,"This app is very easy to use , nice graphics ,...",pranab,0.599167,empty3
3,beaaa17d-44af-44c8-ba38-24ba54c3b17d,83.0,2.42,Quite good but felt slow response from cc. Ove...,pranab,0.366667,empty3
4,89d8c4f0-6fe0-4389-b1f8-913ba894c0f1,74.0,2.33,Not getting otp for login since yesterday. Als...,pranab,0.000000,empty3


In [0]:
p1=p1.drop(columns={'Review Text','Review Title'})
p2=p2.drop(columns={'Review Text','Review Title'})

In [0]:
# Managing the NaN
p1['App Version Code']= p1['App Version Code'].fillna('empty1')
p1['App Version Name']= p1['App Version Name'].fillna('empty2')
p2['App Version Code']= p2['App Version Code'].fillna('empty1')
p2['App Version Name']= p2['App Version Name'].fillna('empty2')

In [0]:
#join two p1 and p2
p2['Star Rating']=0
result = p1.append([p2],sort=True)

In [0]:
result.tail(3)

,App Version Code,App Version Name,Star Rating,id,senti,senti_2
1421,75,2.34,0,a2a7be3d-41ce-472c-9e1e-eca354eedd0f,0.70,empty3
1422,28,1.9,0,af96d36d-42a1-4c8a-9e01-6ac1b2fbee5e,-0.25,empty3
1423,90,2.49,0,0c30ae30-d3c9-4a6d-b6b3-ff17210839f5,-0.20,empty3


In [0]:
#converting the data types
result['App Version Code']=result['App Version Code'].astype(str)
result['App Version Name']=result['App Version Name'].astype(str)

In [0]:
# One hot encoding into columns using dummy variables.
p1_copy=result
dummy1=pd.get_dummies(result['App Version Code'])
dummy2=pd.get_dummies(result['App Version Name'])
dummy3=pd.get_dummies(result['senti_2'])
df_1=pd.concat([p1_copy, dummy1,dummy2,dummy3], axis=1)
df_1=df_1.drop(columns={'App Version Code','App Version Name','senti_2'})
df_1.head()


,Star Rating,id,senti,1.0,10.0,11.0,13.0,14.0,17.0,18.0,...,2.49,2.5,2.6,2.7,2.9,empty2,empty3,happy,neutral,sad
0,1,b817b0f0-a2f8-4c9d-a5f6-d3fbf8b1d7e9,-0.91,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1,c7844e8f-56c1-487b-ae3e-df2fdf4c1767,0.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,93ff57f7-9e02-4fa9-b779-3db82b8af0a4,-0.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,df2dcdef-c09a-4f35-afab-e1231d3fec9a,0.60,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5,11f8f968-4cec-4424-8427-9709ab05b5be,0.65,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [0]:
# Separating the Data
df_f1=df_1[df_1['Star Rating']==0]
df_f2=df_1[df_1['Star Rating']!=0]

In [0]:
df_f2.head()

,Star Rating,id,senti,1.0,10.0,11.0,13.0,14.0,17.0,18.0,...,2.49,2.5,2.6,2.7,2.9,empty2,empty3,happy,neutral,sad
0,1,b817b0f0-a2f8-4c9d-a5f6-d3fbf8b1d7e9,-0.91,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1,c7844e8f-56c1-487b-ae3e-df2fdf4c1767,0.00,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,2,93ff57f7-9e02-4fa9-b779-3db82b8af0a4,-0.20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,5,df2dcdef-c09a-4f35-afab-e1231d3fec9a,0.60,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,5,11f8f968-4cec-4424-8427-9709ab05b5be,0.65,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [0]:
# Shuffleing the training data 
df_f2=df_f2.reindex(np.random.permutation(df_f2.index))

# 2. Creating the synthetic data to balance the dataset and training the model.

In [0]:
# Spliting the Data
x = df_f2.drop(columns={'id','Star Rating'})
y = df_f2['Star Rating']

In [0]:
!pip install smote_variants

    100% |████████████████████████████████| 133kB 3.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/f8/32/ba/ca4264f6b795502698c8db8b07011a953e5fe34457a49c0860
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
Successfully built minisom statistics


In [0]:
# Creating the Synthetic Data
oversampler= sv.MulticlassOversampling(sv.distance_SMOTE())
X_samp, y_samp= oversampler.sample(x,y)

2019-04-18 16:53:13,397:INFO:MulticlassOversampling: Running multiclass oversampling with strategy equalize_1_vs_many_successive
2019-04-18 16:53:13,405:INFO:MulticlassOversampling: Sampling minority class with label: 1
2019-04-18 16:53:13,417:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1}")
2019-04-18 16:53:13,817:INFO:MulticlassOversampling: Sampling minority class with label: 4
2019-04-18 16:53:13,828:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 0.4416427889207259, 'n_neighbors': 5, 'n_jobs': 1}")
2019-04-18 16:53:13,954:INFO:MulticlassOversampling: Sampling minority class with label: 3
2019-04-18 16:53:13,968:INFO:distance_SMOTE: Running sampling via ('distance_SMOTE', "{'proportion': 0.3164971355080089, 'n_neighbors': 5, 'n_jobs': 1}")
2019-04-18 16:53:14,065:INFO:MulticlassOversampling: Sampling minority class with label: 2
2019-04-18 16:53:14,076:INFO:distance_SMOTE: Running samplin

In [0]:
# Dividing the Dataset for training and testing
X_train, X_test, y_train, y_test = train_test_split(X_samp,y_samp, random_state=0, test_size=0.25)

In [0]:
# Fitting the model for training
model=RandomForestClassifier(criterion= 'gini',min_samples_split=15,n_estimators=50)
model.fit(X_train,y_train)
model.score(X_train,y_train)
#Predict Output
predicted= model.predict(X_test)
accuracy_score(y_true = y_test, y_pred = predicted)

0.7099069512862616

### Classification Report and Confusion Matrix

In [0]:
print(metrics.classification_report(y_test, predicted))
print(metrics.confusion_matrix(y_test, predicted))

              precision    recall  f1-score   support

           1       0.67      0.79      0.72       716
           2       0.83      0.80      0.82       734
           3       0.73      0.72      0.72       752
           4       0.67      0.63      0.65       747
           5       0.66      0.61      0.64       705

   micro avg       0.71      0.71      0.71      3654
   macro avg       0.71      0.71      0.71      3654
weighted avg       0.71      0.71      0.71      3654

[[564  39  35  42  36]
 [ 73 585  35  24  17]
 [ 78  42 542  44  46]
 [ 56  21  75 471 124]
 [ 75  14  60 124 432]]


In [0]:
# Output a pickle file for the model
joblib.dump(model, '/content/drive/My Drive/Colab Notebooks/niki/saved_model1.pkl') 

['/content/drive/My Drive/Colab Notebooks/niki/saved_model1.pkl']

In [0]:
scores = cross_val_score(
 model, X_samp, y_samp, cv=5)

In [0]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() / 2))

Accuracy: 0.71 (+/- 0.03)


# 3. Creating the Submission File.

In [0]:
x_test=df_f1.drop(columns={'id','Star Rating'})
predicted2= model.predict(x_test)
final=df_f1.drop(columns={'Star Rating'})
final_dict={'id':df_f1['id'],'Star Rating':predicted2}
final_df=pd.DataFrame(final_dict)
final_df.to_csv("/content/drive/My Drive/Colab Notebooks/niki/sample_submission.csv",index=False)


In [0]:
final_df.head(10)

,id,Star Rating
0,bdcb3129-afc1-4608-825f-558fe9c17e2b,5
1,7518d5c8-5e35-45e2-b29d-cf0996ba9e2f,1
2,b21ca913-ba44-471b-91e1-aaf035379a84,5
3,beaaa17d-44af-44c8-ba38-24ba54c3b17d,4
4,89d8c4f0-6fe0-4389-b1f8-913ba894c0f1,1
5,16e57197-186e-44ef-ac4c-6f6a13e787ee,1
6,04a33480-0544-433d-af9d-e8c0777cadfb,5
7,5a5b705a-e3d8-4424-989c-05d078b6a461,5
8,00aad18b-1960-464c-9334-0961a9f1d65c,4
9,ce2a2f18-bfba-4430-afc5-be587a190d77,2


# Thank You